In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from collections import defaultdict
from sklearn.utils import shuffle
import os
import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd




In [2]:
import numpy as np
import os

# Define the folder path
folder_path = r"D:\PYTHONIG\newwindow\NOTEBOOKS_2025\aprilmay2025\data\numpy\optunadata\STFT"

# Load the numpy files into the respective arrays with the correct capitalized naming
eeg_fold_1 = np.load(os.path.join(folder_path, 'STFT_DATA_FOLD_fold_1.npy'))
labels_fold_1 = np.load(os.path.join(folder_path, 'STFT_LABELS_FOLD_fold_1.npy'))
patients_fold_1 = np.load(os.path.join(folder_path, 'STFT_PATIENTS_FOLD_fold_1.npy'))

eeg_fold_2 = np.load(os.path.join(folder_path, 'STFT_DATA_FOLD_fold_2.npy'))
labels_fold_2 = np.load(os.path.join(folder_path, 'STFT_LABELS_FOLD_fold_2.npy'))
patients_fold_2 = np.load(os.path.join(folder_path, 'STFT_PATIENTS_FOLD_fold_2.npy'))

eeg_fold_3 = np.load(os.path.join(folder_path, 'STFT_DATA_FOLD_fold_3.npy'))
labels_fold_3 = np.load(os.path.join(folder_path, 'STFT_LABELS_FOLD_fold_3.npy'))
patients_fold_3 = np.load(os.path.join(folder_path, 'STFT_PATIENTS_FOLD_fold_3.npy'))

eeg_fold_4 = np.load(os.path.join(folder_path, 'STFT_DATA_FOLD_fold_4.npy'))
labels_fold_4 = np.load(os.path.join(folder_path, 'STFT_LABELS_FOLD_fold_4.npy'))
patients_fold_4 = np.load(os.path.join(folder_path, 'STFT_PATIENTS_FOLD_fold_4.npy'))

eeg_fold_5 = np.load(os.path.join(folder_path, 'STFT_DATA_FOLD_fold_5.npy'))
labels_fold_5 = np.load(os.path.join(folder_path, 'STFT_LABELS_FOLD_fold_5.npy'))
patients_fold_5 = np.load(os.path.join(folder_path, 'STFT_PATIENTS_FOLD_fold_5.npy'))

eeg_folds = [eeg_fold_1, eeg_fold_2, eeg_fold_3, eeg_fold_4, eeg_fold_5]
labels_folds = [labels_fold_1, labels_fold_2, labels_fold_3, labels_fold_4, labels_fold_5]
patients_folds = [patients_fold_1, patients_fold_2, patients_fold_3, patients_fold_4, patients_fold_5]

for i in range(len(eeg_folds)):
    eeg_folds[i] = eeg_folds[i].astype(np.float16)

In [3]:
def data_balancer(data, labels, factor):
    # Count the number of samples in each class
    num_class_0 = np.sum(labels == 0)
    num_class_1 = np.sum(labels == 1)
    num_class_2 = np.sum(labels == 2)

    # Find the minimum number of samples across all classes
    min_samples = min(num_class_0, num_class_1, num_class_2)

    # Calculate the number of samples to take from each class
    samples_per_class = min_samples // factor

    # Randomly sample 'samples_per_class' from each class
    class_0_indices = np.random.choice(np.where(labels == 0)[0], samples_per_class, replace=False)
    class_1_indices = np.random.choice(np.where(labels == 1)[0], samples_per_class, replace=False)
    class_2_indices = np.random.choice(np.where(labels == 2)[0], samples_per_class, replace=False)

    # Combine balanced indices
    balanced_indices = np.concatenate((class_0_indices, class_1_indices, class_2_indices))

    # Shuffle the balanced indices
    np.random.shuffle(balanced_indices)

    # Create balanced training data and labels
    balanced_data = data[balanced_indices]
    balanced_labels = labels[balanced_indices]

    return balanced_data, balanced_labels

In [4]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Initializes the early stopping mechanism based on divergence detection.

        Args:
            patience (int): Number of consecutive epochs with increasing validation loss
                            before stopping.
        """
        self.patience = patience
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Checks if the validation loss is diverging and updates the state accordingly.

        Args:
            val_loss (float): Current epoch's validation loss.
            model (torch.nn.Module): The model being trained.
        """
        if self.best_loss is None or val_loss < self.best_loss:
            # Improvement detected
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            # Validation loss increased
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Divergence detected. Stopping training after {self.counter} epochs.")
                self.early_stop = True

    def load_best_model(self, model):
        """
        Restores the model to the state with the lowest validation loss.

        Args:
            model (torch.nn.Module): The model to restore.
        """
        model.load_state_dict(self.best_model_state)




In [5]:
import torch
import torch.nn as nn

debug_mode_flag = False
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set random seed for reproducibility
torch.manual_seed(42)
debug_mode_flag = False
import torch
import torch.nn as nn

class LstmAttention(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.3):
        super(LstmAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout if num_layers > 1 else 0
        )

        self.attention = nn.MultiheadAttention(embed_dim=hidden_size * 2, num_heads=2, batch_first=True)

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_size * 2, num_classes)
        self.act = nn.Sigmoid() if num_classes == 1 else nn.Softmax(dim=1)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        lstm_out, _ = self.lstm(x)  # (batch, seq, hidden*2)

        # Apply attention: Q=K=V=lstm_out
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)  # (batch, seq, hidden*2)

        # Aggregate via mean pooling or use only last time step
        pooled = attn_out.mean(dim=1)  # (batch, hidden*2)

        out = self.dropout(pooled)
        out = self.fc(out)
        return self.act(out)


In [6]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# For spectrogram shape (224, 224, 3) reshaped to (224, 672)
# 224 time steps, 672 features per time step
input_size = 224 * 3  # 672
hidden_size = 128
num_layers = 1
num_classes = 3

model = LstmAttention(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, num_classes=num_classes)
model.to(device)

# Test with correct input dimensions: (batch_size, seq_len, input_size)
# We need to reshape our data from (batch, 224, 224, 3) to (batch, 224, 672)
x = torch.randn(8, 224, 672).to(device)  # batch_size=8, seq_len=224, features=672
output = model(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")  # Expect: (8, 3)

Input shape: torch.Size([8, 224, 672])
Output shape: torch.Size([8, 3])


In [7]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# pca = sklearn.decomposition.PCA(3)
epochs = 100


fold_indices = np.arange(5)
fold_indices = np.random.permutation(fold_indices)
val_fold_indices = np.roll(fold_indices, 1)

input_shape = 224*3
num_classes = 3


In [ ]:
import optuna
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import balanced_accuracy_score

def objective(trial):
    # Hyperparameter suggestions
    hidden_size = trial.suggest_categorical("hidden_size", [32,64,128,256,512,1024])
    num_layers = trial.suggest_categorical("num_layers", [1, 2, 3, 4])
    dropout = trial.suggest_float("dropout", 0.1, 0.5, step=0.1)

    learning_rate = trial.suggest_float("learning_rate", 1e-8, 1e-2, log=True)
    optimizer_name = "Adam"
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [10, 16, 32])
    label_smoothing = 0.3
    factor = 1

    print(f"trial:{trial.number} hidden_size: {hidden_size}, num_layers: {num_layers}, learning_rate: {learning_rate}, optimizer_name: {optimizer_name}, weight_decay: {weight_decay}, batch_size: {batch_size}, label_smoothing: {label_smoothing}, factor: {factor}")

    # Run only for 1 fold (fold 0)
    test_fold_idx = 0
    test_fold = fold_indices[test_fold_idx]
    remaining_folds = [fold_indices[i] for i in range(5) if i != test_fold_idx]
    val_fold_idx = test_fold_idx % 4
    val_fold = remaining_folds[val_fold_idx]
    train_folds = [fold for fold in remaining_folds if fold != val_fold]

    # Prepare data with correct reshaping
    train_data = np.concatenate([eeg_folds[j] for j in train_folds])
    train_data = train_data.reshape(train_data.shape[0], train_data.shape[1], train_data.shape[2]*train_data.shape[3])
    train_labels = np.concatenate([labels_folds[j] for j in train_folds])

    val_data = eeg_folds[val_fold]
    val_data = val_data.reshape(val_data.shape[0], val_data.shape[1], val_data.shape[2]*val_data.shape[3])
    val_labels = labels_folds[val_fold]

    test_data = eeg_folds[test_fold]
    test_data = test_data.reshape(test_data.shape[0], test_data.shape[1], test_data.shape[2]*test_data.shape[3])
    test_labels = labels_folds[test_fold]

    balanced_train_data, balanced_train_labels = data_balancer(train_data, train_labels, factor=factor)

    # Create data loaders
    train_dataset = TensorDataset(torch.tensor(balanced_train_data, dtype=torch.float32),
                                  torch.tensor(balanced_train_labels, dtype=torch.long))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)

    val_dataset = TensorDataset(torch.tensor(val_data, dtype=torch.float32),
                                torch.tensor(val_labels, dtype=torch.long))
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True)

    test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float32),
                                 torch.tensor(test_labels, dtype=torch.long))
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=1, pin_memory=True)

    # Initialize model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = LstmAttention(
        input_size=672,  # Fixed input size: 224*3
        hidden_size=hidden_size,
        num_layers=num_layers,
        num_classes=num_classes,
        dropout=dropout
    ).to(device)  # Move model to device immediately after creation
    
    criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
    optimizer_cls = {"Adam": optim.Adam, "AdamW": optim.AdamW, "SGD": optim.SGD}
    optimizer = optimizer_cls[optimizer_name](model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    early_stopping = EarlyStopping(patience=10)

    epochs = 30
    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for val_inputs, val_labels in val_loader:
                val_inputs, val_labels = val_inputs.to(device, non_blocking=True), val_labels.to(device, non_blocking=True)
                val_outputs = model(val_inputs)
                loss = criterion(val_outputs, val_labels)
                val_loss += loss.item()

        val_loss /= len(val_loader)

        early_stopping(val_loss, model)
        if early_stopping.early_stop:
            print(f"Early stopping at epoch {epoch}")
            break

    early_stopping.load_best_model(model)

    # Evaluation
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = balanced_accuracy_score(all_labels, all_preds)
    print(f"Trial {trial.number}: Test Accuracy = {accuracy:.4f}")

    del model
    torch.cuda.empty_cache()

    trial.report(accuracy, step=0)

    if trial.should_prune():
        raise optuna.TrialPruned()

    return accuracy

# Start Optuna Study
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)
)

study.optimize(objective, n_trials=300, n_jobs=3) 

# Best result
print("Best hyperparameters:", study.best_params)


c:\Users\Gabriel\anaconda3\envs\cudaenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-06-03 22:18:57,504] A new study created in memory with name: no-name-caa4473a-a634-4667-b7c7-156fed3d9ff1


trial:0 hidden_size: 32, num_layers: 2, learning_rate: 0.0017072686796075181, optimizer_name: Adam, weight_decay: 6.591950956140822e-06, batch_size: 10, label_smoothing: 0.3, factor: 1trial:1 hidden_size: 32, num_layers: 4, learning_rate: 5.4790368880734364e-06, optimizer_name: Adam, weight_decay: 0.00018226544397252048, batch_size: 16, label_smoothing: 0.3, factor: 1
trial:2 hidden_size: 64, num_layers: 4, learning_rate: 1.6372142086864276e-05, optimizer_name: Adam, weight_decay: 8.975823001977633e-06, batch_size: 16, label_smoothing: 0.3, factor: 1



[I 2025-06-03 22:20:41,517] Trial 2 finished with value: 0.5974108961779273 and parameters: {'hidden_size': 64, 'num_layers': 4, 'dropout': 0.1, 'learning_rate': 1.6372142086864276e-05, 'weight_decay': 8.975823001977633e-06, 'batch_size': 16}. Best is trial 2 with value: 0.5974108961779273.


Trial 2: Test Accuracy = 0.5974
trial:3 hidden_size: 1024, num_layers: 2, learning_rate: 7.877736275948196e-08, optimizer_name: Adam, weight_decay: 0.0005039381270787882, batch_size: 10, label_smoothing: 0.3, factor: 1


[I 2025-06-03 22:20:43,043] Trial 1 finished with value: 0.3347701149425288 and parameters: {'hidden_size': 32, 'num_layers': 4, 'dropout': 0.2, 'learning_rate': 5.4790368880734364e-06, 'weight_decay': 0.00018226544397252048, 'batch_size': 16}. Best is trial 2 with value: 0.5974108961779273.


...

[I 2025-06-04 06:09:34,220] Trial 298 finished with value: 0.5670592467993646 and parameters: {'hidden_size': 256, 'num_layers': 4, 'dropout': 0.2, 'learning_rate': 0.0001537122181035339, 'weight_decay': 1.2911800711037775e-06, 'batch_size': 32}. Best is trial 112 with value: 0.7098714138865526.


Trial 298: Test Accuracy = 0.5671
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 20


[I 2025-06-04 06:09:43,274] Trial 299 finished with value: 0.5654681805438745 and parameters: {'hidden_size': 256, 'num_layers': 4, 'dropout': 0.2, 'learning_rate': 0.00015823926337075371, 'weight_decay': 1.0046251309207093e-06, 'batch_size': 32}. Best is trial 112 with value: 0.7098714138865526.


Trial 299: Test Accuracy = 0.5655
Best hyperparameters: {'hidden_size': 64, 'num_layers': 4, 'dropout': 0.30000000000000004, 'learning_rate': 0.0003457860039763819, 'weight_decay': 5.944172232271371e-06, 'batch_size': 32}


: 